In [1]:
#!/public/spark-0.9.1/bin/pyspark

import os
import sys
import csv
import numpy as np
from pynauty import *
import re
import networkx as nx
from hdfs import Config


In [2]:
def get_subgraph_hash(edges):
    # ORDER INSIDE EDGES IS ALREADY DONE
    # NOW ONLY NEED TO ORDER EDGES BETWEEN THEMSELVES
    hash_str = ''
    # rearrange the edges
    edges = sorted(edges)  # sorts on the first elements first and then on second
    for edge in edges:
        hash_str += str(edge[0]) + '.' + str(edge[1]) + ';'
    return hash_str

def map_to_graph(combination, edges):
    # edges are list of tuples

    # this is going to be the key
    original_edges = list()
    if type(combination) is int:
        combination = [combination]
    for ind in combination:
        original_edges.append(edges[ind])
    original_vertices = set()
    for edge in original_edges:
        original_vertices.add(edge[0])
        original_vertices.add(edge[1])
    original_vertices = list(original_vertices)
    g = Graph(len(original_vertices))
    new_edges = list()
    for edge in original_edges:
        g.connect_vertex(original_vertices.index(edge[0]), original_vertices.index(edge[1]))
        new_edges.append((original_vertices.index(edge[0]), original_vertices.index(edge[1])))
    # v_g = VsigramGraph(g, None, label_arr=canon_label(g), orig_edges=original_edges)
    # subgraph_collection = SubgraphCollection(v_g.label_arr, subgraphs=[v_g], freq=1)
    return (canon_label(g), (1, [original_edges]))


def update_subgraph_freq(a, b):
    # freq_object = SubgraphCollection(label=a.label)
    # freq_object.subgraphs = a.subgraphs + b.subgraphs
    # freq_object.freq = a.freq + b.freq
    return (a[0] + b[0], a[1] + b[1])


def filter_by_connected(edges_indexes, orig_edges):
    # create networkx graph
    first_element = True
    edge_new = edges_indexes[len(edges_indexes) - 1]
    edges_old = list(edges_indexes[0:len(edges_indexes) - 1]) if type(edges_indexes[0:len(edges_indexes) - 1]) is list \
        else [edges_indexes[0:len(edges_indexes) - 2]]
    # print str(edges_old) + str(type(edges_old))
    # print str(edge_new) + str(type(edge_new))
    if first_element:
        # print 'INDEXES: ' + str(edges_indexes) + str(type(edges_indexes))
        # print 'OLD: ' + str(edges_old) + str(type(edges_old))
        # print 'NEW: ' + str(edge_new) + str(type(edge_new))
        # print str(edge_new in edges_old)
        first_element = False
    if edge_new in edges_old:
        # print 'REPEATING EDGE'
        return False
    edges_list = list()
    for ind in list(edges_indexes):
        edges_list.append(orig_edges[ind])
    # print edges_list
    nx_g = nx.Graph()
    nx_g.add_edges_from(edges_list)
    return nx.is_connected(nx_g)


def mapToList(edges_indexes):
    tupl_to_list = list(edges_indexes[0]) if type(edges_indexes[0]) is list else [edges_indexes[0]]
    tupl_to_list.append(edges_indexes[1])
    # print 'LIST: ' + str(tupl_to_list)
    return tupl_to_list



In [4]:
from pyspark import SparkContext
from pyspark import SparkConf

In [6]:
# conf = SparkConf().setAppName('SubgraphMining').setMaster('spark://bscdc01.dc.bsc:7077')
# sc = SparkContext(conf=conf)
sc = SparkContext("yarn", "SubgraphMining")
print sc


Deleting HDFS directory...


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=SubgraphMining, master=yarn) created by __init__ at <ipython-input-5-2f179219cc00>:8 

In [9]:
#dfs_root = 'hdfs://bscdc01:54310/'
client = Config().get_client('dev')
print 'Deleting HDFS directory...'
client.delete('subgraphs', recursive=True)
sample = '7d734d06-f2b1-4924-a201-620ac8084c49'

# try to read from hdfs
lines = sc.textFile('samples/new_assignment_separate.csv')
header = lines.first()  # extract header
lines = lines.filter(lambda row: row != header)   # filter out header
positions_rdd = lines.map(lambda line: line.split(','))
# print positions_rdd.collect()
positions_combined = lines.flatMap(lambda line: line.split(','))
positions_distinct = positions_combined.distinct()
positions_distinct_list = positions_distinct.collect()
edges_rdd = positions_rdd.map(lambda positions: [positions_distinct_list.index(positions[0])
                                                 if positions_distinct_list.index(positions[0]) < positions_distinct_list.index(positions[1])
                                                 else positions_distinct_list.index(positions[1]),
                                                 positions_distinct_list.index(positions[1])
                                                 if positions_distinct_list.index(positions[0]) < positions_distinct_list.index(positions[1])
                                                 else positions_distinct_list.index(positions[0])])
edges_rdd = edges_rdd.map(lambda edge: (str(edge[0]) + ':' + str(edge[1]), (edge, 1)))
# print edges_rdd.collect()
edges_rdd = edges_rdd.reduceByKey(lambda edge_kv1, edge_kv2: (edge_kv1[0], edge_kv1[1] + edge_kv2[1]) )
edges_rdd_list = edges_rdd.collect()
edges = [tuple(item[1][0]) for item in edges_rdd_list]
print len(edges)

rdd_1 = sc.parallelize(range(len(edges)))
rdd_of_graphs_1 = rdd_1.map(lambda combination: map_to_graph(combination, edges))
#     rdd_filtered = rdd_of_graphs.filter(lambda x: x[1] is not None)
counts_by_label_1 = rdd_of_graphs_1.reduceByKey(lambda a, b: update_subgraph_freq(a, b))
counts_by_label_list_1 = counts_by_label_1.collect()
counts_by_label_1.saveAsTextFile('subgraphs/' + sample + '/' + str(1))
rdd_last = rdd_1
print rdd_1.count()

for i in range(2, 4):
    print 'SIZE ' + str(i)
    # for each element in rdd_1 create a list and add to new rdd
    rdd_next = rdd_last.cartesian(rdd_1)
    print rdd_next.first()
    # rdd_next = rdd_next.flatMap(lambda x: [element for tupl in x for element in tupl])
    # filter the connected graphs
    rdd_next = rdd_next.map(lambda x: mapToList(x))
    rdd_next = rdd_next.filter(lambda comb: filter_by_connected(comb, edges))

    print rdd_next.first()
    rdd_last = rdd_next
    # create a graph from each list
    rdd_of_graphs = rdd_next.map(lambda combination: map_to_graph(combination, edges))
#     rdd_filtered = rdd_of_graphs.filter(lambda x: x[1] is not None)
    counts_by_label = rdd_of_graphs.reduceByKey(lambda a, b: update_subgraph_freq(a, b))
    counts_by_label.saveAsTextFile('subgraphs/' + sample + '/' + str(i))

Deleting HDFS directory...
578
578
SIZE 2
(0, 0)
[0, 18]
SIZE 3
([0, 18], 0)
[1, 2, 6]


In [10]:
sc.stop()